<a href="https://colab.research.google.com/github/anik007-code/Cv_AI_scoring/blob/main/cv_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import PyPDF2
from collections import Counter
import spacy

nlp = spacy.load("en_core_web_sm")


def extract_text(pdf):
    with open(pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

path = "/content/drive/MyDrive/CV/Simple Resume of Md Arifur Rahman Anik.pdf"
# print(extract_text(path))

In [75]:
def word_count(text):
    doc = nlp(text)
    token = [token.text for token in doc]
    words = [word for word in token if word.isalpha()]
    return len(Counter(words))/2 * 0.1
# print(word_count(extract_text(path)))

15.5


In [68]:
def assess_langauge_quality(text):
    doc = nlp(text)
    action_verbs = ["manage", "lead", "develop", "design", "implement", "analyze", "collaborate", "create", "internship"]
    for token in doc:
        if token.pos_ == "VERB" and token.lemma_ in action_verbs:
          print(token.lemma_)
          return len(token)
# print(assess_langauge_quality(extract_text(path)))


In [69]:
def assess_structure(cv_text):
    score = 0
    sections = ["education", "skills", "projects", "certifications", "contact", "summary"]
    for section in sections:
        if section in cv_text.lower():
            score += 5
    return score

# print(assess_structure(extract_text(path)))

In [70]:
import re
from datetime import datetime

def parse_date(date_str):
    """Parse various date formats into a datetime object."""
    formats = [
        '%d/%m/%Y', '%d-%m-%Y', '%d/%m/%y', '%d-%m-%y',  # Common formats
        '%b %Y', '%B %Y',  # Short and full month names with year
        '%d %b %Y', '%d %B %Y',  # Full day and month names
        '%Y-%m', '%Y/%m',  # Year and month formats
    ]

    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"Date format for '{date_str}' is not recognized.")

def calculate_total_experience(cv_text):
    # Regex pattern to match various date formats in CV
    date_pattern = r'(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?) \d{4}|\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4}|\d{2}/\d{2}/\d{2}|\d{2}-\d{2}-\d{2}|\b\d{4}[-/]\d{2})\s*[-to]*\s*(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?) \d{4}|\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4}|\d{2}/\d{2}/\d{2}|\d{2}-\d{2}-\d{2}|\b\d{4}[-/]\d{2}|Present)'

    dates = []

    # Extract date ranges using regex pattern
    matches = re.findall(date_pattern, cv_text)
    for match in matches:
        if not match:
            continue
        start_date_str = match[0].strip()
        end_date_str = match[1].strip() if len(match) > 1 else "Present"

        try:
            start_date_obj = parse_date(start_date_str)
        except ValueError:
            # print(f"Skipping invalid start date format: {start_date_str}")
            continue  # Skip invalid dates

        if end_date_str.lower() == 'present':
            end_date_obj = datetime.now()
        else:
            try:
                end_date_obj = parse_date(end_date_str)
            except ValueError:
                # print(f"Skipping invalid end date format: {end_date_str}")
                continue  # Skip invalid dates

        # print(f"Start Date: {start_date_obj}, End Date: {end_date_obj}")
        dates.append((start_date_obj, end_date_obj))

    # Calculate total experience
    total_experience = 0
    for start_date, end_date in dates:
        experience = (end_date - start_date).days / 365.25  # Convert days to years
        # print(f"Experience from {start_date} to {end_date}: {experience:.2f} years")
        total_experience += experience

    return total_experience


# Calculate total experience
# total_experience = calculate_total_experience(extract_text(path))
# print(f"Total Experience: {total_experience:.2f}")


def experience_score(text):
    total_experience = calculate_total_experience(text)
    if total_experience >= 5:
        return 20
    elif 3 <= total_experience < 5:
        return 15
    elif 1 <= total_experience < 3:
        return 10
    elif 0.5 <= total_experience < 1:
        return 5
    else:
        return 1

# print(experience_score(extract_text(path)))

In [71]:
programming_languages = ["Python", "Django", "Flask", "FastAPI", "Pyramid", "Bottle", "TensorFlow", "Keras", "PyTorch", "Java", "Spring", "Hibernate",
                        "JavaServer Faces (JSF)", "Apache Struts", "Grails", "C++", "Qt", "Boost", "POCO", "Cocos2d-x", "JavaScript", "React",
                        "Angular", "Vue.js", "Node.js", "Express", "Meteor", "Ember.js", "C#", "ASP.NET", "Entity Framework", "Xamarin", "Blazor",
                        "NancyFX", "Ruby", "Ruby on Rails", "Sinatra", "Hanami", "Padrino", "PHP", "Laravel", "Symfony", "CodeIgniter",
                        "Zend Framework", "Yii", "Swift", "Vapor", "Kitura", "Perfect", "Go", "Gin", "Echo", "Beego", "Revel", "Kotlin", "Ktor",
                        "Spring Boot", "Exposed", "Rust", "Actix", "Rocket", "Tide", "Scala", "Akka", "Play Framework", "Lagom", "TypeScript",
                        "NestJS", "Angular", "Next.js", "R", "Shiny", "R Markdown", "ggplot2", "Perl", "Dancer", "Mojolicious", "Catalyst", "MATLAB",
                        "Simulink", "MATLAB Production Server", "SQL", "SQLAlchemy", "MyBatis", "Hibernate", "HTML", "Bootstrap", "Foundation", "Bulma",
                        "CSS", "Sass", "LESS", "Tailwind CSS", "Dart", "Flutter", "AngularDart", "Shell", "Bash", "Zsh", "Fish", "Objective-C", "Cocoa",
                        "Cocoa Touch", "Visual Basic", "ASP.NET Web Forms", "WinForms", "WPF", "Assembly", "MASM", "NASM", "Haskell", "Yesod", "Snap",
                        "Play", "Lua", "LÖVE", "Corona SDK", "Groovy", "Grails", "Spock", "Ratpack", "SAS", "SAS Studio", "SAS Visual Analytics", "F#",
                        "Fable", "Suave", "Giraffe", "VHDL", "ModelSim", "Vivado", "Verilog", "ModelSim", "Vivado", "ActionScript", "Apache Flex",
                        "Erlang", "Cowboy", "N2O", "Phoenix", "Smalltalk", "Pharo", "Squeak", "Lisp", "Common Lisp", "CLIPS", "Prolog", "SWI-Prolog",
                        "GNU Prolog", "Ada", "GNAT", "RPG", "IBM Rational Developer for i", "Delphi", "Embarcadero Delphi", "FoxPro", "Visual FoxPro",
                        "ML", "OCaml", "SML/NJ", "PostScript", "Ghostscript", "JScript", "ASP.NET Web Forms", "Tcl", "Tk", "Tix", "XSLT", "Saxon",
                        "Xalan", "PowerShell", "PowerShell Core", "Powershell DSC", "Solidity", "Truffle", "Hardhat", "Julia", "Flux", "Knet", "HCL",
                        "Terraform", "PL/SQL", "Oracle SQL Developer", "SML", "Standard ML of New Jersey", "LabVIEW", "NI LabVIEW", "Razor",
                        "ASP.NET Core Razor", "Forth", "Gforth", "Eiffel", "EiffelStudio", "Clojure", "Compojure", "Luminus", "Elixir", "Phoenix",
                        "Nerves", "Xamarin", "Xamarin.Forms", "Apache Groovy", "Grails", "Spock", "Jupyter Notebook", "JupyterLab", "MATLAB/Simulink",
                        "Simulink", "NATURAL", "Adabas NATURAL", "ABAP", "SAP ABAP", "REXX", "Open Object REXX"]
def language_score(text):
    score = 0
    for language in programming_languages:
        if language in extract_text(path):
            score += 2

    return score
print(language_score(extract_text(path)))


20


In [77]:
def ai_cv_score():
    score = 0
    score1 = assess_langauge_quality(extract_text(path))
    score2 = assess_structure(extract_text(path))
    score3 = experience_score(extract_text(path))
    score4 = language_score(extract_text(path))
    score5 = word_count(extract_text(path))
    score = score1 + score2 + score3 + score4 + score5
    if score >= 100:
        return 99
    return score

print(ai_cv_score())

develop
75.5
